In [1]:
import sys
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import os
import inspect
import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../")
sys.path.insert(0, project_path)

from Program.Objects.Human import *
from Program.Objects.Issue import *
from Program.Objects.Machine import *
from Program.Objects.MaintenanceWorkOrder import *
from Program.Objects.Tag import *

from Program.Database.Database_Properties import *
from Program.Database.Database import *
from Program.Database.Main import *

from Program.KPI.KPI import *


from Program.Others.MyDate import *

# DATABASE

## FILES

### NIST server

### Greystone

In [2]:
PassWord = 'GREYSTONE!!'
#file = os.path.join(project_path , 'Data_CSV', 'GreyStone_Data.csv')
file = os.path.join(project_path , 'Data_CSV', 'GS_tokenized.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 15,
                NodeHuman.VALUE_OPERATOR.value: 13,

                NodeTag.VALUE_ITEM.value: 17,
                NodeTag.VALUE_PROBLEM.value: 19,
                NodeTag.VALUE_SOLUTION.value: 20,
                NodeTag.VALUE_NA.value: 18,
                NodeTag.VALUE_UNKNOWN.value: 21,
                NodeTag.VALUE_STOP_WORDS.value: 22,

                NodeMachine.VALUE_MACHINE.value: 7,
                NodeIssue.VALUE_MACHINE_DOWN.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 0,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 1,
                NodeIssue.VALUE_PART_PROCESS.value: 16,
                
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value: 9,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value + "2": 10,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value: 11,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value + "2": 12
                }

def date_cleanizer(date, time="0000"):
    """
    clean the date from GS data

    :param date: the date
    :param time: the time
    :return: a DateTime
    """

####    DATE ####
    if date is "":
        return None

    date = date.replace(" ","").split("/")

    date[2] = "20" + date[2]

    if len(date[0]) is 1:
        date[0] = "0" + date[0]
    else:
        date[0] = date[0]

    if len(date[1]) is 1:
        date[1] = "0" + date[1]
    else:
        date[1] = date[1]

#### TIME ####
    while len(time) < 4:
        time = "0" + time.replace(" ", "")

    return datetime(year=int(date[2]), month=int(date[0]), day=int(date[1]), hour=int(time[:2]), minute=int(time[2:]))

### Psu

In [ ]:
PassWord = 'PSU!!'
file = os.path.join(project_path , 'Data_CSV', 'Psu_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 18,

                NodeMachine.VALUE_MACHINE.value: 3,
                NodeMachine.VALUE_TYPE.value:6,
                NodeMachine.VALUE_MANUFACTURER.value:5,
                NodeMachine.VALUE_LOCASION.value:4,


                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 11,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 19,
                NodeIssue.VALUE_DESCRIPTION_CAUSE.value:12,
                NodeIssue.VALUE_MACHINE_DOWN.value: 13,
                }

### Hvac

In [ ]:
PassWord = 'HVAC!!'
file = os.path.join(project_path , 'Data_CSV', 'Hvac_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 10,
                NodeHuman.VALUE_OPERATOR.value: 43,

                NodeMachine.VALUE_MACHINE.value: 7,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 247,
                }
date_cleanizer = None

### Auto

In [ ]:
PassWord = 'AUTO!!'
file = os.path.join(project_path , 'Data_CSV', 'Auto_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 4,
                NodeHuman.VALUE_OPERATOR.value: 11,
                NodeHuman.VALUE_CRAFTS.value: 16,
                NodeHuman.VALUE_SKILLS.value: 17,

                NodeMachine.VALUE_MACHINE.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 7,
                }

### Mine

In [ ]:
PassWord = 'MINE!!'
file = os.path.join(project_path , 'Data_CSV', 'Mine_Data.csv')

localization = {}

## ACTION

### Connect

In [3]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", PassWord)

In [ ]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", "11/Jul/2017!")

## Indexs

In [ ]:
database.createIndexes()

In [ ]:
database.dropIndexes()

## Data

In [5]:
graph_database_from_csv(database, file, localization, date_cleanizer)

100%|█████████▉| 3438/3448 [00:57<00:00, 60.11it/s]

3438 Maintenance Word Order Created


In [ ]:
# token_dataframe = pd.DataFrame.from_csv("GS_tokenized_forSascha.csv", index_col =None)
# graph_database_from_tag_extractor(database = database, token_dataframe = token_dataframe)

In [4]:
database.deleteData()

1

# KPI

# KPI using Object

In [ ]:
tec = Technician(name=["_"])
ope = Operator(name=["_"])

i3=Issue(problem=["_"], solution=["_"], machine_down="_",date_maintenance_work_order_issue=["_"], date_maintenance_work_order_close=["_"],part_in_process=["_"] )

mac = Machine(name=["_","h11", "h1"])

tagI = TagItem(keyword = ["_"])
tagS = TagAction(keyword = ["_"] , it_is="s")
tagP = TagAction(keyword = ["_"] , it_is="p")



In [ ]:
query = cypher_from_kpi(tec,mac)

In [ ]:
dataframe = pandas_from_cypher_kpi(database, query)

In [ ]:
dataframe.shape

In [ ]:
print(query)

In [ ]:
dataframe

# For UI


In [ ]:
query = "MATCH (node) RETURN DISTINCT labels(node)"
results_label, done = database.runQuery(query)
label_property_dict = dict()

list1 = []
listN = []
for result_label in results_label.records():
    for labels in result_label.values():
        for label in labels:
            if label in listN:
                pass
            elif label in list1:
                list1.remove(label)
                listN.append(label)
            else:
                list1.append(label)

for label in list1:
    print("ttttt")
    query = f'MATCH (node:{label})<-[relation]-() RETURN distinct type(relation)'
    results_relation, done = database.runQuery(query)
    print(results_relation)

for label in listN:
    query = f'MATCH (node:{label}) RETURN keys(node)'
    results_relation, done = database.runQuery(query)
    label_property_dict.update({label: {p for proper in results_relation.records() for prop in
                                        proper.values() for p in prop }})
            


In [ ]:
label_property_dict

In [ ]:
list1

In [ ]:
listN

In [ ]:
dict={"problem" : "(node:ACTION:TAG)<-[:PROBLEM]-(i:ISSUE)",
    "solution" : "(node:ACTION:TAG)<-[:SOLUTION]-(i:ISSUE)",
    "tag" : "(node:TAG)",
    "human" : "(node:HUMAN)",
    "technican" : "(node:TECHNICIAN:HUMAN)<-[:SOLVE_BY]-(i:ISSUE)",
    "operator" : "(node:OPERATOR:HUMAN)<-[:REQUESTED_BY]-(i:ISSUE)",
    "machine" : "(node:MACHINE)",
    "machine_type" : "(node:MACHINE_TYPE)",
     "issue" : "(node:ISSUE)"}

In [ ]:
for key, value in dict.items():
    #print("___")
    query = "MATCH " + value + "RETURN distinct keys(node)"
   # print(query)
    results, done = database.runQuery(query)
    for record in results.records():
        #print("----")
        print(record.values())

### test other ---------

In [11]:
query = "MATCH (node)<-[relationship]-() RETURN DISTINCT labels(node) as node, type(relationship) as rel"
results, done = database.runQuery(query)
label_property_dict = dict()

df = pd.DataFrame(columns=['nodes','edges','properties'])
for result in results.records():
    for node_link in result.values():
        df['node']=

['ITEM', 'TAG']
CONTAINS
---
['ACTION', 'TAG']
PROBLEM
---
['HUMAN', 'TECHNICIAN']
SOLVE_BY
---
['ACTION', 'TAG']
SOLUTION
---
['HUMAN', 'OPERATOR']
REQUESTED_BY
---
['MACHINE']
COVERED
---
['HUMAN', 'OPERATOR', 'TECHNICIAN']
SOLVE_BY
---
['HUMAN', 'OPERATOR', 'TECHNICIAN']
REQUESTED_BY
---


In [12]:
df

,nodes,edges,properties
